# Codificación de las variables categoricas y escalado 

En este paso del análisis, he aplicado técnicas para convertir las variables categóricas en formatos numéricos apropiados para su uso en futuros modelos de machine learning. Para ello, se han utilizado métodos como la Target Encoding y One-Hot Encoding, según la naturaleza de cada variable. Además, hemos llevado a cabo un proceso de escalado para las variables numéricas, con el fin de garantizar que todas las características tengan el mismo rango de valores. Este paso es crucial, especialmente para modelos que son sensibles a las escalas de las variables, como los algoritmos basados en distancias.

## Importo librerias

Para comenzar, se importan las librerías necesarias en este notebook.

In [2]:
import pandas as pd 
import numpy as np
import sklearn
# conda install category_encoders
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

## Lectura de datos del procesado inicial

Es importante señalar que, en este tercer notebook, trabajaremos con el archivo final generado a partir del procesamiento llevado a cabo en el notebook anterior donde ya habiamos realizado una separación en train y test estratificado. 

In [3]:
pd_loan_train = pd.read_csv("../data/interim/train_pd_data_preprocessing_missing_outlier.csv")\
                  .set_index("SK_ID_CURR") 
pd_loan_test = pd.read_csv("../data/interim/test_pd_data_preprocessing_missing_outlier.csv")\
                  .set_index("SK_ID_CURR") 

In [4]:
pd_loan_train.columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       ...
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',
       'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
       'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'TARGET'],
      dtype='object', length=121)

In [5]:
pd_loan_train.dtypes

NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
CNT_CHILDREN                    int64
                               ...   
AMT_REQ_CREDIT_BUREAU_WEEK    float64
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
TARGET                          int64
Length: 121, dtype: object

## Codificación de la variable objetivo

Es importante recordar que la variable objetivo `TARGET` toma el valor 1 para clientes con dificultades de pago (retrasos mayores a X días en al menos una de las primeras Y cuotas) y 0 para todos los demás casos.

In [6]:
pd_loan_train['TARGET'].value_counts()

TARGET
0    226148
1     19860
Name: count, dtype: int64

In [7]:
X_train = pd_loan_train.drop('TARGET',axis=1)
X_test = pd_loan_test.drop('TARGET',axis=1)
y_train = pd_loan_train['TARGET']
y_test = pd_loan_test['TARGET']

## Codificación del resto de variables categoricas

En este trabajo, elijo codificar las variables categóricas tipo string de dos maneras: las variables con pocas categorías (tres o menos) se codifican utilizando One-Hot Encoding, mientras que aquellas con más de tres categorías se codifican mediante Target Encoding.
    
El enfoque de codificación se basa en la cantidad de categorías: One-Hot Encoding es ideal para pocas categorías, ya que evita suposiciones sobre relaciones, mientras que Target Encoding es más eficiente para muchas categorías, al reducir dimensionalidad y capturar patrones relevantes con la variable objetivo.

In [8]:
list_columns_cat = list(X_train.select_dtypes("object", "category").columns)
list_other = list(set(X_train.columns)-set(list_columns_cat))

In [9]:
list_columns_cat

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

Con este objetivo, decido separar las columnas categóricas que contienen cadenas de texto en función de sus valores únicos, para así poder determinar si utilizaré One-Hot Encoding o Target Encoding en cada caso.

In [10]:
def separar_por_unicos(df, list_columns_cat):
    '''
    ----------------------------------------------------------------------------------------------------------
    Función separar_por_unicos:
    ----------------------------------------------------------------------------------------------------------
    - Descripción: 
        Función que recibe un DataFrame y una lista de columnas categóricas y separa las 
        columnas en dos listas según el número de valores únicos que tienen. 
        
    - Inputs:
        - df (DataFrame): Pandas DataFrame que contiene los datos.
        - list_columns_cat (list): Lista con los nombres de las columnas categóricas del dataset.
        
    - Return:
        - list_columns_more_three_cat: Lista con los nombres de las columnas 
          categóricas que tienen más de 3 valores únicos.
        - list_columns_less_three_cat: Lista con los nombres de las columnas 
          categóricas que tienen 3 o menos valores únicos.
    ----------------------------------------------------------------------------------------------------------
    '''
    
    list_columns_more_three_cat = []  
    list_columns_less_three_cat = []  
    
    for col in list_columns_cat:
        num_unicos = df[col].nunique()  # Cuenta el número de valores únicos en la columna
        
        if num_unicos > 3:
            list_columns_more_three_cat.append(col)  
        else:
            list_columns_less_three_cat.append(col)  
    
    return list_columns_more_three_cat, list_columns_less_three_cat

In [11]:
list_columns_more_three_cat, list_columns_less_three_cat = separar_por_unicos(X_train, list_columns_cat)

In [12]:
list_columns_more_three_cat

['NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE']

In [13]:
list_columns_less_three_cat

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'EMERGENCYSTATE_MODE']

Comenzamos codificando con One-Hot Encoding, ideal para variables con pocas categorías.

In [14]:
ohe = ce.OneHotEncoder(cols=list_columns_less_three_cat)
model_ohe = ohe.fit(X_train, y_train)

In [15]:
model_ohe

OneHotEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
                    'FLAG_OWN_REALTY', 'EMERGENCYSTATE_MODE'])

In [16]:
X_train_ohe = model_ohe.transform(X_train, y_train)
X_test_ohe = model_ohe.transform(X_test, y_test)

In [17]:
X_train_ohe

,NAME_CONTRACT_TYPE_1,NAME_CONTRACT_TYPE_2,CODE_GENDER_1,CODE_GENDER_2,CODE_GENDER_3,FLAG_OWN_CAR_1,FLAG_OWN_CAR_2,FLAG_OWN_REALTY_1,FLAG_OWN_REALTY_2,CNT_CHILDREN,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
412054,1,0,1,0,0,1,0,1,0,1,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
291454,0,1,1,0,0,1,0,0,1,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
245085,1,0,1,0,0,0,1,0,1,3,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
180128,0,1,1,0,0,1,0,1,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,9.0
390833,0,1,1,0,0,1,0,1,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265043,0,1,1,0,0,1,0,1,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
102945,0,1,1,0,0,1,0,0,1,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
392425,0,1,1,0,0,1,0,1,0,0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0



Tenemos que tener en cuenta que One-Hot Encoding aumenta la dimensionalidad del conjunto de datos, ya que crea una nueva columna por cada categoría única en la variable.

Continuamos codificando con Target Encoding, ideal para variables con muchas categorías.

In [18]:
te = ce.TargetEncoder(cols=list_columns_more_three_cat)  
model_te = te.fit(X_train_ohe, y_train)  

In [19]:
model_te

TargetEncoder(cols=['NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
                    'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
                    'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
                    'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE',
                    'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE',
                    'WALLSMATERIAL_MODE'])

In [20]:
X_train_t = model_te.transform(X_train_ohe) 
X_test_t = model_te.transform(X_test_ohe, y_test)

In [21]:
X_train_t

,NAME_CONTRACT_TYPE_1,NAME_CONTRACT_TYPE_2,CODE_GENDER_1,CODE_GENDER_2,CODE_GENDER_3,FLAG_OWN_CAR_1,FLAG_OWN_CAR_2,FLAG_OWN_REALTY_1,FLAG_OWN_REALTY_2,CNT_CHILDREN,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
412054,1,0,1,0,0,1,0,1,0,1,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
291454,0,1,1,0,0,1,0,0,1,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
245085,1,0,1,0,0,0,1,0,1,3,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
180128,0,1,1,0,0,1,0,1,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,9.0
390833,0,1,1,0,0,1,0,1,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265043,0,1,1,0,0,1,0,1,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
102945,0,1,1,0,0,1,0,0,1,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
392425,0,1,1,0,0,1,0,1,0,0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0


Por otro lado, Target Encoding no aumenta la dimensionalidad, ya que reemplaza cada categoría por la media del objetivo correspondiente, lo que lo hace más eficiente para variables con muchas categorías.

In [22]:
X_train_t.dtypes.to_dict()

{'NAME_CONTRACT_TYPE_1': dtype('int64'),
 'NAME_CONTRACT_TYPE_2': dtype('int64'),
 'CODE_GENDER_1': dtype('int64'),
 'CODE_GENDER_2': dtype('int64'),
 'CODE_GENDER_3': dtype('int64'),
 'FLAG_OWN_CAR_1': dtype('int64'),
 'FLAG_OWN_CAR_2': dtype('int64'),
 'FLAG_OWN_REALTY_1': dtype('int64'),
 'FLAG_OWN_REALTY_2': dtype('int64'),
 'CNT_CHILDREN': dtype('int64'),
 'AMT_INCOME_TOTAL': dtype('float64'),
 'AMT_CREDIT': dtype('float64'),
 'AMT_ANNUITY': dtype('float64'),
 'AMT_GOODS_PRICE': dtype('float64'),
 'NAME_TYPE_SUITE': dtype('float64'),
 'NAME_INCOME_TYPE': dtype('float64'),
 'NAME_EDUCATION_TYPE': dtype('float64'),
 'NAME_FAMILY_STATUS': dtype('float64'),
 'NAME_HOUSING_TYPE': dtype('float64'),
 'REGION_POPULATION_RELATIVE': dtype('float64'),
 'DAYS_BIRTH': dtype('float64'),
 'DAYS_EMPLOYED': dtype('float64'),
 'DAYS_REGISTRATION': dtype('float64'),
 'DAYS_ID_PUBLISH': dtype('float64'),
 'OWN_CAR_AGE': dtype('float64'),
 'FLAG_MOBIL': dtype('int64'),
 'FLAG_EMP_PHONE': dtype('int64'

## Escalado de las variables

Una vez que las variables son numéricas, algunos algoritmos requieren que estén escaladas. Esto implica ajustar los valores para que estén en una misma escala. Para ello, utilizamos el StandardScaler de sklearn, que normaliza los datos. Algoritmos como SVM, KNN y Regresión logística son sensibles a la escala de los datos, y si las variables tienen magnitudes muy diferentes, el rendimiento del modelo puede verse afectado.

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
model_scaled = scaler.fit(X_train_t)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_t), columns=X_train_t.columns, index=X_train_t.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_t), columns=X_test_t.columns, index=X_test.index)

In [24]:
X_train_scaled.describe()

,NAME_CONTRACT_TYPE_1,NAME_CONTRACT_TYPE_2,CODE_GENDER_1,CODE_GENDER_2,CODE_GENDER_3,FLAG_OWN_CAR_1,FLAG_OWN_CAR_2,FLAG_OWN_REALTY_1,FLAG_OWN_REALTY_2,CNT_CHILDREN,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,246008.000000,2.460080e+05,2.460080e+05,2.460080e+05,...,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05
mean,-5.499306e-17,-1.253518e-16,-2.483208e-17,-7.870593e-18,-5.009380e-20,1.179867e-16,0.000000,4.684808e-17,-4.684808e-17,-8.087215e-17,...,-2.216763e-17,1.222830e-17,1.839480e-17,-3.899193e-19,1.587116e-17,-7.379584e-18,2.538808e-17,-3.660909e-17,-4.280448e-17,1.680985e-17
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-3.242183e-01,-3.084341e+00,-1.391379e+00,-7.186917e-01,-3.492116e-03,-1.395292e+00,-0.716696,-6.641440e-01,-1.505698e+00,-5.781722e-01,...,-9.055708e-02,-2.445197e-02,-2.209133e-02,-1.848158e-02,-7.143397e-02,-5.993498e-02,-1.573323e-01,-2.704609e-01,-2.942555e-01,-1.007229e+00
25%,-3.242183e-01,3.242183e-01,-1.391379e+00,-7.186917e-01,-3.492116e-03,-1.395292e+00,-0.716696,-6.641440e-01,-1.505698e+00,-5.781722e-01,...,-9.055708e-02,-2.445197e-02,-2.209133e-02,-1.848158e-02,-7.143397e-02,-5.993498e-02,-1.573323e-01,-2.704609e-01,-2.942555e-01,-4.409132e-01
50%,-3.242183e-01,3.242183e-01,7.187112e-01,-7.186917e-01,-3.492116e-03,7.166960e-01,-0.716696,-6.641440e-01,6.641440e-01,-5.781722e-01,...,-9.055708e-02,-2.445197e-02,-2.209133e-02,-1.848158e-02,-7.143397e-02,-5.993498e-02,-1.573323e-01,-2.704609e-01,-2.942555e-01,-4.409132e-01
75%,-3.242183e-01,3.242183e-01,7.187112e-01,1.391417e+00,-3.492116e-03,7.166960e-01,1.395292,1.505698e+00,6.641440e-01,8.069321e-01,...,-9.055708e-02,-2.445197e-02,-2.209133e-02,-1.848158e-02,-7.143397e-02,-5.993498e-02,-1.573323e-01,-2.704609e-01,-2.942555e-01,6.917185e-01
max,3.084341e+00,3.242183e-01,7.187112e-01,1.391417e+00,2.863593e+02,7.166960e-01,1.395292,1.505698e+00,6.641440e-01,2.573881e+01,...,1.104276e+01,4.089650e+01,4.526662e+01,5.410792e+01,5.082886e+01,9.183782e+01,4.247521e+01,2.779571e+01,3.340640e+02,1.201804e+01


El escalado de variables ha sido exitoso, ya que cada columna presenta una media cercana a 0 y una desviación estándar de 1, lo que facilitará la comparación entre características al situarlas en una misma escala.

## Guardado de la tabla

Ahora, tras un tercer procesamiento y análisis de los datos, se guarda el DataFrame para conservar este nuevo estado proceso y facilitar su uso en futuross modelos. 

In [ ]:
X_train_scaled.to_csv("../data/processed/train_pd_data_scaled.csv")
X_test_scaled.to_csv("../data/processed/train_pd_data_scaled.csv")

# Conclusiones


Antes de comenzar con las conclusiones, quiero aclarar que, aunque es una buena práctica guardar las funciones automatizadas en un archivo externo o notebook auxiliar para importarlas cuando sea necesario, he decidido no hacerlo en esta primera práctica. Considero que esto podría interrumpir el flujo de trabajo y afectar la claridad del análisis, por lo que he optado por definir cada función en el momento en que la necesite. En futuras prácticas, utilizaré un notebook auxiliar para gestionar estas funciones de manera más eficiente.

En resumen, he realizado un análisis exploratorio del conjunto de datos para entender las variables y su relación con el incumplimiento de pago. He analizado la distribución de las variables numéricas y categóricas utilizando herramientas visuales como histogramas y boxplots. Además, dividí el dataset en subconjuntos de entrenamiento y test, e identifiqué y traté los valores nulos y outliers.

He examinado las correlaciones entre las variables utilizando métodos estadísticos como Pearson, V de Cramer y WOE, y transformé las variables categóricas mediante One-Hot Encoding y Target Encoding. Finalmente, apliqué técnicas de escalado a las variables numéricas para prepararlas adecuadamente para el desarrollo de modelos predictivos.

Tras este primer análisis exploratorio, he observado que sí existe un perfil de cliente más propenso a no devolver un préstamo. En general, los clientes con más hijos tienden a enfrentar mayores dificultades para cumplir con los pagos, lo que sugiere que la carga familiar puede afectar la capacidad de pago.

En cuanto a los tipos de ingresos, los desempleados y aquellos en baja por maternidad presentan un mayor riesgo de impago. Por otro lado, empresarios y estudiantes parecen estar menos expuestos a este riesgo, lo que se refleja en los valores de WOE, que indican una relación inversa con las dificultades de pago.

La ocupación también es un factor clave: los clientes en trabajos de baja cualificación (como los Low-Skill Laborers) tienen más dificultades para pagar sus préstamos, mientras que aquellos en ocupaciones más cualificadas, como gerentes o contadores, presentan un menor riesgo de impago.

El nivel educativo también influye significativamente, ya que los clientes con secundaria incompleta tienen una mayor probabilidad de impago, en comparación con aquellos con niveles educativos más altos, quienes suelen tener una mayor estabilidad financiera.

Finalmente, el tipo de préstamo también juega un papel importante. Los préstamos revolventes están asociados con un menor riesgo de impago, mientras que los préstamos en efectivo tienen una relación más débil con las dificultades de pago.

En resumen, los clientes más propensos a no devolver un préstamo son aquellos con condiciones laborales o económicas desfavorables, como los desempleados, las personas con trabajos de baja cualificación, aquellos con muchos hijos y los que tienen un nivel educativo bajo. Estos factores pueden ser claves para identificar el riesgo de impago en futuros análisis.

Quiero mencionar que estas hipótesis serán contrastadas en futuras prácticas al desarrollar el modelo de aprendizaje supervisado. No obstante, he podido formularlas gracias al análisis realizado en los notebooks anteriores, mediante gráficas de barras y boxplots, el cálculo de la V de Cramér, y el uso de Weight of Evidence (WOE) e Information Value (IV).



# Práctica 1: Análisis Exploratorio de Datos

**Autor:** Gorka Zubiri Elso

**Correo electrónico:** gorka.zubiri@cunef.edu

**Directorio GitHub:** Gorka Zubiri Elso

Este proyecto tiene como objetivo ayudar a un banco a mejorar la aprobación de préstamos mediante el análisis de datos históricos de solicitudes. La idea es identificar patrones en los clientes que cumplen con sus pagos y aquellos que no lo hacen, para aplicar esta información en futuros modelos predictivos.

## Objetivos principales de este trabajo

1. **Explorar y entender el dataset**: Familiarizarme con las variables que contiene el conjunto de datos.
2. **Análisis de las variables**: Realizar un análisis general de las variables y su distribución, con especial atención a su relación con la variable objetivo (dificultad de pago).
3. **Depuración de los datos**: Limpiar y preparar el dataset para la aplicación de modelos predictivos, gestionando adecuadamente los valores nulos, outliers y tipos de datos erróneos.
4. **Preprocesamiento para Machine Learning**: Codificar las variables categóricas y escalar las variables numéricas para que los datos estén listos para futuros modelos.
5. **Análisis descriptivo**: Extraer conclusiones del análisis exploratorio que permitan entender los componentes clave del dataset y su relación con la variable objetivo.

## Análisis Exploratorio de Datos (EDA)

Este análisis tiene como propósito entender las principales características del conjunto de datos y cómo estas se relacionan con la probabilidad de incumplimiento de pago. Los resultados obtenidos en este análisis son los siguientes:

### 1. Comprensión de las variables

A través de un análisis preliminar, se ha logrado entender el significado y la distribución de cada una de las variables presentes en el dataset.

### 2. Estudio de la distribución y naturaleza de los datos

Se realizó un análisis detallado de la distribución de las variables tanto numéricas como categóricas. Además, se utilizaron visualizaciones gráficas (como histogramas, diagramas de dispersión y boxplots) para observar la distribución de las variables y sus relaciones con la variable objetivo.

### 3. División del dataset en train y test

Para garantizar una correcta evaluación de los modelos, se realizó una división estratificada del dataset en dos subconjuntos: entrenamiento (train) y prueba (test).

### 4. Identificación de valores nulos y outliers

Durante la exploración de los datos, se identificaron variables con valores faltantes y outliers. Se tomaron decisiones adecuadas para el tratamiento de estos problemas, considerando la naturaleza de cada variable y su posible impacto en el análisis.

### 5. Correlaciones entre variables

Se analizaron las correlaciones entre las variables numéricas y categóricas utilizando herramientas estadísticas como el coeficiente de Pearson para las variables continuas, y el coeficiente V de Cramer y Weight of Evidence (WOE) para las variables categóricas, entre otros.

### 6. Codificación de variables categóricas

Para transformar las variables categóricas en un formato numérico adecuado para modelos de machine learning, se aplicaron técnicas como One-Hot Encoding y Target Encoding.

### 7. Escalado de variables numéricas

Finalmente, se aplicaron técnicas de escalado a las variables numéricas para normalizar su rango y asegurar que todas las características tengan la misma importancia en el análisis.

## Estructura del Directorio

La estructura del directorio de este proyecto está organizada de la siguiente manera:

- **`data/`**: Contiene los archivos de datos con los que vamos a trabajar.

  - **`raw/`**: Archivos de datos originales, tal como se obtuvieron.
  
  - **`processed/`**: Datos que ya han sido procesados y transformados para su uso.
  
  - **`interim/`**: Datos intermedios que han sido parcialmente procesados y aún no están listos para su uso final.
  
  
- **`docs/`**: Carpeta que contiene documentos relacionados con el proyecto, como hojas de datos o documentación explicativa.


- **`env/`**: Archivos relacionados con el entorno de desarrollo, incluyendo un archivo `requirements.txt` con todas las librerías y dependencias utilizadas en el proyecto.
- **`notebooks/`**: Contiene los notebooks en formato Jupyter (`.ipynb`) que documentan el análisis de datos y otros experimentos.


- **`html/`**: Carpeta donde se almacenan los notebooks convertidos en formato HTML para facilitar su visualización y compartición.


- **`images/`**: Directorio que alberga las imágenes generadas en los notebooks, como gráficas y diagramas.


- **`model/`**: Contiene los modelos entrenados y sus archivos correspondientes. Aquí se guardan los modelos para su posterior uso o evaluación.


- **`src/`**: Directorio que guarda los archivos fuente de Python, tales como scripts, funciones o clases utilizadas en el procesamiento de datos o la creación de modelos.

## Notebooks Desarrollados

Se han desarrollado tres notebooks en esta primera práctica:

1. **01_Exploracion_General**: Su objetivo es comprender las variables y estudiar la distribución y naturaleza de los datos.
2. **02_EDA_Procesamiento_Vars**: Enfocado en la división del dataset en train y test, la identificación de valores nulos y outliers, y el análisis de correlaciones entre variables.
3. **03_Codificacion_Var_Escalado**: Se centra en la codificación de variables categóricas y el escalado de variables numéricas.